In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.functions import col,sum,avg,max

In [0]:
df_parquet = spark.read.format("parquet").option("header",True).load("/mnt/Gen2/Customer/parquetFiles")

In [0]:
display(df_parquet.show(10))

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
C_CUSTKEY| C_NAME| C_ADDRESS|C_NATIONKEY| C_PHONE|C_ACCTBAL|C_MKTSEGMENT| C_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
 34136|Customer#000034136|UjBY ZyJnHesu0bxL...| 0|10-863-612-2949| 3397.87| BUILDING|es. furiously flu...|
 31731|Customer#000031731|Zq9KcCB7jMCIGMj32...| 0|10-250-546-6987| 8466.07| FURNITURE|counts lose quick...|
 33511|Customer#000033511| bz2D0HrwQiaz| 0|10-618-323-6263| 5331.22| AUTOMOBILE|ons are slyly ins...|
 45695|Customer#000045695|Oc2Yin5Gcqudb6RGE...| 0|10-414-601-9544| 5340.52| AUTOMOBILE|riously furious p...|
 36768|Customer#000036768| ,8Q8TZ48Wme| 0|10-950-600-3087| 9229.21| MACHINERY|ounts engage quic...|
 37449|Customer#000037449|kSezsusukDqm37pMN...| 0|10-231-238-1522| 1173.53| AUTOMOBILE|ding dependencies...|
 46970|Customer#000046970| uZbY1HQYjGobtk8ppkV| 0|10-361-617-6664| 8689.85| AUTOMOBILE|xes boost furious...|
 29109|Customer#000029109|FG6PVFkL69,LZWcl4...| 0|10-770-489-7404| 1939.54| AUTOMOBILE| above the furiou...|
 43753|Customer#000043753|xWtHn81QCu6kSjIkd...| 0|10-180-882-4254| 4975.15| AUTOMOBILE|sts-- idly final ...|
 42429|Customer#000042429|g6AgY9ecZcwMhWvq5...| 0|10-616-749-7989| 5345.32| BUILDING|ing to the slyly ...|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
only showing top 10 rows

In [0]:
for i in range(1,20):
  df_parquet.write.format("delta").mode("append").save("/mnt/Gen2/CustomerDelta")

In [0]:
#Creating CustomerDelta table
spark.sql("CREATE TABLE CustomerDelta USING DELTA LOCATION '/mnt/Gen2/CustomerDelta/'")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-2253846730212391> in <cell line: 2>()
      1 #Creating CustomerDelta table
----> 2 spark.sql("CREATE TABLE CustomerDelta USING DELTA LOCATION '/mnt/Gen2/CustomerDelta/'")

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/session.py in sql(self, sqlQuery, **kwargs)
   1117             sqlQuery = formatter.format(sqlQuery, **kwargs)
   1118         try:
-> 1119             return DataFrame(self._jsparkSession.sql(sqlQuery), self)
   1120         finally:
   1121             if len(kwa

In [0]:
for i in range(1,20):
  df_parquet.write.format("delta").partitionBy("C_MKTSEGMENT").mode("append").save("/mnt/Gen2/CustomerDeltaPartition")

In [0]:
#Creating CustomerDeltaPartition table
#spark.sql("DROP TABLE CustomerDeltaPartition")
spark.sql("CREATE TABLE CustomerDeltaPartition USING DELTA LOCATION '/mnt/Gen2/CustomerDeltaPartition/'")

Out[19]: DataFrame[]

In [0]:
%sql
-- Updating non partitioned table
-- Immediately go to the Notebook 6.4.2.ConcurrencyTest and run the first cell (DELETE FROM CustomerDelta WHERE C_MKTSEGMENT='FURNITURE')
UPDATE CustomerDelta SET C_NATIONKEY = 4 WHERE C_MKTSEGMENT='MACHINERY'

num_affected_rows
1138062


In [0]:
%sql
-- Updating non partitioned table
-- Immediately go to the Notebook 6.4.2.ConcurrencyTest and run the second cell (DELETE FROM CustomerDeltaPartition WHERE C_MKTSEGMENT='FURNITURE')
UPDATE CustomerDeltaPartition SET C_NATIONKEY = 4 WHERE C_MKTSEGMENT='MACHINERY'

num_affected_rows
1138062


In [0]:
%sql

ALTER TABLE CustomerDeltaPartition SET TBLPROPERTIES ('delta.isolationLevel' = 'Serializable')

In [0]:
# After running this cell, Immediately go to the Notebook 6.4.2.ConcurrencyTest and run the third cell (DELETE FROM CustomerDeltaPartition WHERE C_MKTSEGMENT='MACHINERY' AND C_CUSTKEY=1377)
for i in range(1,20):
  df_parquet.where(col("C_MKTSEGMENT")=="MACHINERY").write.format("delta").mode("append").partitionBy("C_MKTSEGMENT").save("/mnt/Gen2/CustomerDeltaPartition")


In [0]:
%sql
ALTER TABLE CustomerDeltaPartition SET TBLPROPERTIES ('delta.isolationLevel' = 'WriteSerializable')